# Determination of polarization

```{autolink-concat}
```

Given the aligned polarimeter field $\vec\alpha$ and the corresponding intensity distribution $I_0$, the intensity distribution $I$ for a polarized decay can be computed as follows:

$$
I\left(\phi,\theta,\chi; \tau\right) = I_0(\tau)\left(1+\vec{P} R(\phi,\theta,\chi) \vec{\alpha}(\tau)\right)
$$ (eq:master.intensity)

with $R$ the rotation matrix over the decay plane orientation, represented in Euler angles $\left(\phi, \theta, \chi\right)$. In this section, we show that it's possible to determine the polarization $\vec{P}$ from a given toy study intensity distribution $I$.

In [ ]:
from __future__ import annotations

import logging
import os
from warnings import filterwarnings

import iminuit
import jax.numpy as jnp
import matplotlib.pyplot as plt
import numpy as np
from tensorwaves.interface import DataSample, Function
from tqdm.auto import tqdm

from polarimetry import formulate_polarimetry
from polarimetry.amplitude import AmplitudeModel, DalitzPlotDecompositionBuilder
from polarimetry.data import create_data_transformer, generate_phasespace_sample
from polarimetry.io import (
    mute_jax_warnings,
    perform_cached_doit,
    perform_cached_lambdify,
)
from polarimetry.lhcb import load_model_builder, load_model_parameters
from polarimetry.lhcb.particle import load_particles
from polarimetry.plot import use_mpl_latex_fonts

filterwarnings("ignore")
mute_jax_warnings()
logging.getLogger("tensorwaves.data").setLevel(logging.ERROR)

NO_TQDM = "EXECUTE_NB" in os.environ
if NO_TQDM:
    logging.getLogger().setLevel(logging.ERROR)

In [ ]:
def load_model(
    file: str, model_choice: int | str = 0, reference_subsystem: int = 1
) -> tuple[AmplitudeModel, DalitzPlotDecompositionBuilder]:
    particles = load_particles("../../data/particle-definitions.yaml")
    builder = load_model_builder(file, particles, model_choice)
    parameters = load_model_parameters(file, builder.decay, model_choice, particles)
    model = builder.formulate(reference_subsystem, cleanup_summations=True)
    model.parameter_defaults.update(parameters)
    return model, builder


MODEL, BUILDER = load_model("../../data/model-definitions.yaml", model_choice=0)
POLARIMETRY_EXPRS = formulate_polarimetry(BUILDER, reference_subsystem=1)
UNFOLDED_POLARIMETRY_EXPRS = [
    perform_cached_doit(expr.doit().xreplace(MODEL.amplitudes))
    for expr in tqdm(POLARIMETRY_EXPRS, disable=NO_TQDM, leave=False)
]
UNFOLDED_INTENSITY_EXPR = perform_cached_doit(MODEL.full_expression)

In [ ]:
POLARIMETRY_FUNCS = [
    perform_cached_lambdify(expr.xreplace(MODEL.parameter_defaults), backend="jax")
    for expr in tqdm(UNFOLDED_POLARIMETRY_EXPRS, disable=NO_TQDM, leave=False)
]
INTENSITY_FUNC = perform_cached_lambdify(
    UNFOLDED_INTENSITY_EXPR.xreplace(MODEL.parameter_defaults),
    backend="jax",
)

The phase space sample is uniformly generated over the Dalitz plane variables $\tau$. In addition, we extend the phase space samples with a uniform distribution over the decay plane angles $\left(\phi, \theta, \chi\right)$.

In [ ]:
N_EVENTS = 100_000
# Dalitz variables
PHSP = generate_phasespace_sample(MODEL.decay, N_EVENTS, seed=0)
PHSP.update(create_data_transformer(MODEL)(PHSP))
# Decay plane variables
PHSP["phi"] = np.random.uniform(-np.pi, +np.pi, N_EVENTS)
PHSP["cos_theta"] = np.random.uniform(-1, +1, N_EVENTS)
PHSP["chi"] = np.random.uniform(-np.pi, +np.pi, N_EVENTS)

We now generate an intensity distribution over the phase space sample given a certain value for $\vec{P}$ using Eq.&nbsp;{eq}`eq:master.intensity`:

In [ ]:
def compute_polarized_intensity(
    Px: float,
    Py: float,
    Pz: float,
    intensity_func: Function[DataSample, jnp.ndarray],
    polarimetry_funcs: tuple[
        Function[DataSample, jnp.ndarray],
        Function[DataSample, jnp.ndarray],
        Function[DataSample, jnp.ndarray],
    ],
    phsp: DataSample,
) -> jnp.array:
    P = jnp.array([Px, Py, Pz])
    R = compute_rotation_matrix(phsp)
    alpha = jnp.array([func(phsp).real for func in polarimetry_funcs])
    I0 = intensity_func(phsp)
    return I0 * (1 + jnp.einsum("i,ij...,j...->...", P, R, alpha))


def compute_rotation_matrix(phsp: DataSample) -> jnp.ndarray:
    ϕ = phsp["phi"]
    θ = jnp.arccos(phsp["cos_theta"])
    χ = phsp["chi"]
    return jnp.einsum("ki...,ij...,j...->k...", Rz(ϕ), Ry(θ), Rz(χ))


def Rz(angle: jnp.ndarray) -> jnp.ndarray:
    N_EVENTS = len(angle)
    ones = jnp.ones(N_EVENTS)
    zeros = jnp.zeros(N_EVENTS)
    return jnp.array(
        [
            [+jnp.cos(angle), -jnp.sin(angle), zeros],
            [+jnp.sin(angle), +jnp.cos(angle), zeros],
            [zeros, zeros, ones],
        ]
    )


def Ry(angle: jnp.ndarray) -> jnp.ndarray:
    N_EVENTS = len(angle)
    ones = jnp.ones(N_EVENTS)
    zeros = jnp.zeros(N_EVENTS)
    return jnp.array(
        [
            [+jnp.cos(angle), zeros, +jnp.sin(angle)],
            [zeros, ones, zeros],
            [-jnp.sin(angle), zeros, +jnp.cos(angle)],
        ]
    )

In [ ]:
P = (+0.22, +0.01, -0.67)
I = compute_polarized_intensity(*P, INTENSITY_FUNC, POLARIMETRY_FUNCS, PHSP)
I /= jnp.mean(I)  # normalized times N for log likelihood

In [ ]:
plt.rc("font", size=18)
use_mpl_latex_fonts()

fig, axes = plt.subplots(figsize=(15, 4), ncols=3)
fig.tight_layout()
for ax in axes:
    ax.set_yticks([])
axes[0].hist(PHSP["phi"], weights=I, bins=80)
axes[1].hist(PHSP["cos_theta"], weights=I, bins=80)
axes[2].hist(PHSP["chi"], weights=I, bins=80)
axes[0].set_xlabel(R"$\phi$")
axes[1].set_xlabel(R"$\cos\theta$")
axes[2].set_xlabel(R"$\chi$")
plt.show()

fig, ax = plt.subplots(figsize=(12, 3))
ax.hist2d(PHSP["sigma2"], PHSP["sigma1"], weights=I, bins=100, cmin=1)
ax.set_xlabel(R"$\sigma_2$")
ax.set_ylabel(R"$\sigma_1$")
ax.set_aspect("equal")
plt.show()

:::{note}
In this example, the intensity and alpha functions are exact amplitude models, but they can be substituted by the **interpolation functions** that use the exported polarimeter fields from {ref}`uncertainties:Exported distributions` as input.  This removes the need to formulate a complete amplitude model in order to determine polarization. See {ref}`appendix/serialization:Import and interpolate` for how to interpolate these exported distributions over the Dalitz plane.
:::

The distribution is assumed to be a measured distribution where $\vec{P}$ is unknown. It is shown below that the actual $\vec{P}$ with which the distribution was generated can be found by performing a fit on Eq.&nbsp;{eq}`eq:master.intensity`. This is done with [`iminuit`](https://iminuit.rtfd.io), starting with a certain 'guessed' value for $\vec{P}$ as initial parameters.

In [ ]:
P_guess = (+0.3, -0.3, +0.4)

In [ ]:
progress_bar = tqdm(desc="Performing fit")


def weighted_nll(Px: float, Py: float, Pz: float) -> float:
    I_new = compute_polarized_intensity(
        Px, Py, Pz, INTENSITY_FUNC, POLARIMETRY_FUNCS, PHSP
    )
    I_new /= jnp.sum(I_new)
    progress_bar.update()
    estimator_value = -jnp.sum(jnp.log(I_new) * I)
    progress_bar.set_postfix_str(
        f"NLL={estimator_value:.4g}, Px={Px:+.3f}, Py={Py:+.3f}, Pz={Pz:+.3f}"
    )
    return estimator_value


optimizer = iminuit.Minuit(weighted_nll, *P_guess)
optimizer.errordef = optimizer.LIKELIHOOD
fit_result = optimizer.migrad()
fit_result

The 'fitted' $\vec{P}$ is equal to the original $\vec{P}$ up to 4 decimals.

In [ ]:
P_fit = np.array([p.value for p in fit_result.params])
np.testing.assert_array_almost_equal(P_fit, P, decimal=4)